# Import library

In [ ]:
# basic modules 
import numpy as np # linear algebra
import geopandas as gpd
import pandas as pd 
import json 
import glob 
import os 

# visualizaztion modules 
from bokeh.resources import INLINE 
from bokeh.io import output_notebook, show, output_file 
from bokeh.plotting import figure 
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, CustomJS, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import brewer,mpl
from bokeh.transform import factor_cmap

# interactive visualization models 
from bokeh.io import curdoc, output_notebook 
from bokeh.models import Slider, HoverTool 
from bokeh.layouts import widgetbox, row, column

output_notebook()

# World Map GeoJSON

In [ ]:
#Read shapefile using Geopandas
gdf2 = gpd.read_file('../input/countries-geojson/ne_10m_admin_0_countries.geojson')
#gdf2.head()

# Converting JSON to DataFrame

In [ ]:
xs = []
ys = []
for obj in gdf2.geometry.boundary:
    if obj.type == 'LineString':
        obj_x, obj_y = obj.xy
        xs.append([[list(obj_x)]])
        ys.append([[list(obj_y)]])
    elif obj.type == 'MultiLineString':
        obj_x = []
        obj_y = []
        for line in obj:
            line_x, line_y = line.xy
            obj_x.append([list(line_x)])
            obj_y.append([list(line_y)])
        xs.append(obj_x)
        ys.append(obj_y)

Country = gdf2['ADMIN'].values        

map_plot = pd.DataFrame({'Country': Country, 'xs': xs, 'ys': ys})
#map_plot.head(5)

# World Happiness Score Data

In [ ]:
#Read df_2016
df = pd.read_csv('../input/fixed-2016-happiness/2016.csv')
#df.head(5)

# Data Merge: Country Boundaries and Happiness Score

In [ ]:
#Merge dataframes gdf and df_2016.
merged2 = map_plot.merge(df, how='inner', on='Country')
#merged2.head(5)

# Choropleth Map

In [ ]:
#Create a shared datasource
source = ColumnDataSource(merged2)

#Define a sequential multi-hue color palette.
palette = brewer['YlGn'][5]
# palette = mpl['Viridis'][5]

#Reverse color order so that dark blue is highest happiness score.
palette = palette[::-1]

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
choro_color_mapper = LinearColorMapper(palette = palette, low = 3, high = 8)

#Define custom tick labels for color bar.
tick_labels = {'3': '3', '4': '4', '5':'5', '6':'6', '7':'7', '8':'8'}

#Add hover tool
hover = HoverTool(tooltips = [ ('Country','@Country'),('Region', '@Region'), ('Happiness Score', '@{Happiness Score}')])
TOOLS="tap, pan, wheel_zoom, zoom_in, zoom_out, box_select"

#Create color bar. 
color_bar = ColorBar(color_mapper=choro_color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal', major_label_overrides = tick_labels)

#Create figure object.
p1 = figure(title = 'World Happiness by Country in 2016', plot_height = 650 , plot_width = 1100, tools = [hover, TOOLS])
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
#Add patch renderer to figure. 
p1.multi_polygons('xs','ys', source = source, fill_color = {'field' :'Happiness Score', 'transform' : choro_color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)
p1.axis.visible = False

#Specify figure layout.
p1.add_layout(color_bar, 'below')

#Display figure.
show(p1)

# Scatter Plot for Happiness and Relevant Scores

In [ ]:
from bokeh.models import CustomJS, RadioButtonGroup

#Color mapper for scatter
scatter_color_mapper = CategoricalColorMapper(factors=df['Region'].unique(), palette=brewer['Spectral'][10])

tooltip_options = [[("Country ", "@Country"), ("Economy (GDP per Capita)", "@{Economy (GDP per Capita)}"), ('Happiness Score', '@{Happiness Score}'),], 
                   [("Country ", "@Country"), ("Family", "@{Family}"), ('Happiness Score', '@{Happiness Score}'),],
                   [("Country ", "@Country"), ("Health (Life Expectancy)", "@{Health (Life Expectancy)}"), ('Happiness Score', '@{Happiness Score}'),],
                   [("Country ", "@Country"), ("Freedom", "@{Freedom}"), ('Happiness Score', '@{Happiness Score}'),],
                   [("Country ", "@Country"), ("Trust (Government Corruption)", "@{Trust (Government Corruption)}"), ('Happiness Score', '@{Happiness Score}'),],
                   [("Country ", "@Country"), ("Generosity", "@{Generosity}"), ('Happiness Score', '@{Happiness Score}'),]
                  ]

#Hover Tool
hover = HoverTool(tooltips=tooltip_options[0])

p2 = figure(tools=[hover, TOOLS], title="Scatter Plot of GDP & Happiness Score Index in 2016")

#Create the scatter
circle = p2.circle('Economy (GDP per Capita)', 'Happiness Score', source=source, fill_color = {'field' :'Region', 'transform' : scatter_color_mapper}, size=15, legend_field='Region',)
p2.xaxis.axis_label = 'Economy (GDP per Capita)'
p2.yaxis.axis_label = 'Happiness Score'
p2.legend.location = "bottom_right"
p2.width=1100
p2.height=600

#Create Radio Button Group
LABELS = ['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity']
radio_button_group = RadioButtonGroup(labels=LABELS, active=0)
radio_button_group.js_on_change('active', CustomJS(args=dict(circle=circle, axis=p2.xaxis[0], title=p2.title, hover=p2.hover, options=tooltip_options), code="""
    circle.glyph.x = {field: cb_obj.labels[cb_obj.active]};
    axis.axis_label = cb_obj.labels[cb_obj.active];
    title.text = "Scatter Plot of " + cb_obj.labels[cb_obj.active] + " & Happiness Score Index in 2016";
    hover[0].tooltips = options[cb_obj.active];
"""))

layout=column(radio_button_group, p2)
show(layout)  

# Interactive Dashboard

In [ ]:
from bokeh.layouts import gridplot

p = gridplot([[p1], [radio_button_group],[p2]])

show(p)